In [ ]:
import pandas as pd
import os
import time
import datetime
import math
import configparser
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

config_file = '../config.ini'
config = configparser.ConfigParser()
config.read(config_file)

y_test_ref = {"tp":1, "fp":0, "fn":1, "tn":0}
y_pred_ref = {"tp":1, "fp":1, "fn":0, "tn":0}

In [ ]:
def convert_confusion_matrix(y_test, y_pred, row):
    for i in ['tp', 'fp', 'tn', 'fn']:
        for j in range(int(row[i])):
            y_test.append(y_test_ref[i])
            y_pred.append(y_pred_ref[i])
            
def get_y_score(y_score, matches):
    for i in matches:
        tmp = eval(i)
        for j in tmp:
            y_score.append(j[1]/100)
            #y_score.append((j[1]/100)/5)
            #y_score.append(1/6896)
            
def get_y_true(y_test, y_pred):
    return [x == y_pred[i] and x == True for i,x in enumerate(y_test)]

def plot_roc_curve(fpr, tpr, scorer):
    plt.plot(fpr, tpr, color='orange', label='ROC for %s' % scorer)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - ' + scorer)
    plt.legend()
    plt.show()

In [ ]:
#scorer_title = ["Levenshtein", "Damerau-Levenshtein", "Jaro", "Jaro-Winkler", "Hamming"]
scorer_title = ["Ratio", "Partial Ratio", "Token Set Ratio", "Partial Token Set Ratio",
                "Token Sort Ratio", "Partial Token Sort Ratio"]
for i, scorer in enumerate(scorer_title):
    df = pd.read_excel(os.path.join('../output/method-fuzzywuzzy-results.xlsx'), sheet_name=scorer)
    df.fillna(value='0', inplace=True)
    y_test = []
    y_pred = []
    y_score = []
    df_records = df.to_dict('records')
    [convert_confusion_matrix(y_test, y_pred, x) for x in df_records]
    print('\n\n', scorer, '\n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    y = get_y_true(y_test, y_pred)
    get_y_score(y_score, df.matches)
    (fpr, tpr, thresholds) = roc_curve(y, y_score, drop_intermediate=False)
    #print(y, y_score)
    #print(fpr, tpr, thresholds)
    plot_roc_curve(fpr, tpr, scorer)
    print('ROC AUC:', roc_auc_score(y, y_score))

In [ ]:
scorer_title = ["2-grams Cosine", "2-grams Jaccard", "2-grams Dice", "2-grams Overlap Coefficient", "2-grams Tversky Index", "3-grams Cosine", "3-grams Jaccard", "3-grams Dice"]
for i, scorer in enumerate(scorer_title):
    df = pd.read_excel(os.path.join('../output/method-token-based-results.xlsx'), sheet_name=scorer)
    df.fillna(value='0', inplace=True)
    y_test = []
    y_pred = []
    y_score = []
    df_records = df.to_dict('records')
    [convert_confusion_matrix(y_test, y_pred, x) for x in df_records]
    print('\n\n', scorer, '\n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    y = get_y_true(y_test, y_pred)
    get_y_score(y_score, df.matches)
    (fpr, tpr, thresholds) = roc_curve(y, y_score)
    plot_roc_curve(fpr, tpr, scorer)
    print('ROC AUC:', roc_auc_score(y, y_score))